In [ ]:
import h5py
import pysam

import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix,csc_matrix

import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import os
import anndata

In [ ]:
DataDir = '/file/path/prefix/'
positive_ad = sc.read(os.path.join(DataDir,'Mouse_Female_5wCells_17wPeaks.h5ad'))
positive_ad

In [ ]:
positive_ad.var_names

In [ ]:
#create negative sample matrix

In [ ]:
nega = csc_matrix(arg1=(50040,87919))
nega = sc.AnnData(nega)

In [ ]:
nega.var_names

In [ ]:
nega_peak = pd.read_csv('./nega_femalemus.csv')
nega_peak

In [ ]:
nega.var_names = nega_peak['windows_negative_subset'].values

In [ ]:
nega.var['features'] = nega.var_names

In [ ]:
nega.obs_names = positive_ad.obs_names

In [ ]:
nega.obs

In [ ]:
positive_ad.obs

In [ ]:
positive_ad.shape, nega.shape

In [ ]:
merge_pos_nega = anndata.concat([positive_ad,nega],join='outer',axis = 1)
merge_pos_nega

In [ ]:
merge_pos_nega.var

In [ ]:
1.shuffle

In [ ]:
## shuffle ad.var
import random

def shuffleData(ad, seed=666):
    random.seed(seed)
    index = [i for i in range(len(ad.var))]
    random.shuffle(index)
    ad_shuffled = ad[:,index]
    return ad_shuffled

In [ ]:
ad = shuffleData(merge_pos_nega)
ad

In [ ]:
ad.var

In [ ]:
feature = ad.var['features'].str.split(pat='-', n=-1, expand=True).rename(columns={0:'chr', 1:'start', 2:'end'}) 
feature['features'] = feature.chr+'-'+feature.start+'-'+feature.end
feature

In [ ]:
ad.var['chr'] = feature['chr'].values
ad.var['start'] = feature['start'].values
ad.var['end'] = feature['end'].values

In [ ]:
ad.var

In [ ]:
peaks = ad.var.iloc[:,1:4]
peaks

In [ ]:
#start to pre datasets

In [ ]:
#ad.copy()
#y = ad.X.T.toarray()
#y[:5,:5],y.shape

In [ ]:
#one-hot

In [ ]:
onehot_nuc = {'A':[1,0,0,0],
            'C':[0,1,0,0],
            'G':[0,0,1,0],
            'T':[0,0,0,1],
            'N':[0,0,0,0]}
            
def _onehot_seq(seq):
    return np.array([onehot_nuc[nuc] for nuc in str(seq).upper()])

In [ ]:
# genome
genome = pysam.Fastafile('/file/path/prefix/ref/mm10/mm10.fa')
genome

In [ ]:
seq_onehot = []

for peak in tqdm(peaks.values):
    seqnames, start, end = peak[:3]
    #chrom = str(seqnames.replace("chr",""))
    start, end = int(start), int(end)
    chrom = seqnames
    # catch overflowed error
    chrom_size = genome.get_reference_length(chrom)
    if end >= chrom_size:
        print(peak[-1])
        pad = 'N' * (end - chrom_size) # pad N
        end = chrom_size
    # fetch sequence 
    seq = genome.fetch(reference=chrom, start=start, end=end)
    # pad N
    if start + 500 > chrom_size:
        seq += pad
    # onehot    
    seq = _onehot_seq(seq)
    seq_onehot.append(seq)

seq_onehot = np.array(seq_onehot, dtype=np.bool)
genome.close()

seq_onehot.shape

In [ ]:
#.Split train/valid/test data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
idx = peaks.index.values
mask = peaks.chr.astype(str) == 'chr8'

test_idx = idx[mask]
train_val_idx = idx[~mask]

train_idx, val_idx = train_test_split(train_val_idx, test_size=0.1)

train_idx.shape, val_idx.shape, test_idx.shape

In [ ]:
peaks['train_test_split'] = "train"
peaks.loc[train_idx, 'train_test_split'] = "train"
peaks.loc[val_idx, 'train_test_split'] = "val"
peaks.loc[test_idx, 'train_test_split'] = "test"

peaks.head()

In [ ]:
# create coordinate sparse matrix
pmat_co = coo_matrix(ad.X.T)

In [ ]:
pmat_co.shape

In [ ]:
## 5.Write
compress_args = {'compression': 'gzip', 'compression_opts': 1}

h5file = h5py.File('Femalemus_5wCells_17wPeaks_8wnegative.shuffled.noimpute.500bp.20230822.h5', 'a')
h5file.create_dataset("pmat/X", data=seq_onehot, dtype=bool, **compress_args)
h5file.create_dataset("pmat/peak", data=peaks.values.astype(np.bytes_), **compress_args)

# h5file.create_dataset("pmat/pmat_sc/y", data=y, dtype=np.float32, **compress_args)
h5file.create_dataset("pmat/pmat_sc/i", data=pmat_co.row, dtype=np.int32, **compress_args)
h5file.create_dataset("pmat/pmat_sc/j", data=pmat_co.col, dtype=np.int32, **compress_args)
h5file.create_dataset("pmat/pmat_sc/x", data=pmat_co.data, dtype=np.int32, **compress_args)
h5file.create_dataset("pmat/pmat_sc/dim", data=pmat_co.shape, dtype=np.int32, **compress_args)

# h5file.create_dataset("cellanno", data=anno.values.astype(np.bytes_), **compress_args)

h5file.close()